In [3]:
import numpy as np
import math as m
import itertools
import scipy as sp

def binary_decode(bytearray: list[int]):
    bytearray = np.flip(bytearray)
    return sum(bytearray * (2 ** np.array(list(enumerate(bytearray.flat)))[:,0]))

def binary_encode(ingest: list[int], num_bits: int):
    return np.floor_divide(ingest, (2 ** np.array(list(enumerate(range(0,(num_bits)))))[:,0])) % 2

def func(i):
    return m.tanh(i)

def float_func(i, precision, ancillary):
    num_bits = precision+ancillary
    val = i * (2**(precision))
    #print(val)
    bin_val = binary_encode(val,num_bits)
    return np.array(np.flip(bin_val))

def float_decomp(i, precision, ancillary):
    num_bits = precision+ancillary
    exponent = np.array([0]*ancillary + list(range(1,precision+1)))
    div = np.array([2]*(num_bits))
    div = div ** exponent
    div = np.array([1]*(num_bits)) / div
    return np.sum(div*i)


def build_unitary(precision, ancillary):

    num_bits = precision+ancillary

    bin_inputs = np.array(list(itertools.product([0, 1], repeat=(precision))))

    bin_inputs = np.unique(np.pad(bin_inputs,(ancillary,0)), axis=0)

    inputs = np.apply_along_axis(float_decomp, 1, bin_inputs,precision,ancillary)

    index_inputs =  np.array(list(enumerate(inputs.tolist())))[:,0].astype(int)

    outputs = np.array([func(i) for i in inputs])

    bin_outputs = np.array([float_func(o,precision,ancillary) for o in outputs])

    index_outputs = np.array([binary_decode(bo) for bo in bin_outputs]).astype(int)

    output_vectors_matrix = sp.sparse.csr_matrix((np.ones(index_outputs.shape),(index_inputs,index_outputs)),(int(2**num_bits),(2**num_bits))).toarray()

    return (np.linalg.qr(output_vectors_matrix)[0], output_vectors_matrix)


def apply_unitary(unitary, dec_input, precision, ancillary):

    num_bits = precision + ancillary

    input = float_func(dec_input, precision, ancillary)

    input_basis_state = np.zeros(2**num_bits)

    input_basis_state[int(binary_decode(input))] = 1

    output_container = input_basis_state.copy()

    for u in unitary:
        output_container = output_container @ u

    #output_basis_state = np.abs(input_basis_state @ unitary)

    output_basis_state = output_container.copy()

    prediction_indexes = np.array(output_basis_state.nonzero())

    probs = output_basis_state[prediction_indexes]

    #output_basis_state=np.abs(output_basis_state)

    max_probs = np.argwhere(output_basis_state == np.amin(output_basis_state))

    predictions = np.array([float_decomp(np.flip(binary_encode(prediction_index,num_bits)),precision,ancillary) for prediction_index in prediction_indexes[0,:]])

    top_predictions = np.array([float_decomp(np.flip(binary_encode(max_prob,num_bits)),precision,ancillary) for max_prob in max_probs])
    
    #print(probs)
    #print(predictions)

    return np.min(top_predictions)

precision = 8
ancillary = 0

num_bits = precision + ancillary

unitary,basic_op = build_unitary(precision,ancillary)

left,sigma,right = np.linalg.svd(basic_op)
right = right @ np.diag((sigma.astype(int)*-1))

num_tests = 90

test_inputs = np.random.rand(num_tests)

predictions = np.array([apply_unitary([left,right],predict,precision,ancillary) for predict in test_inputs])

actuals = np.array([func(real) for real in (test_inputs)])

np.max(np.abs(predictions - actuals))


0.006964669018210044

In [4]:
print(left.nonzero()[0].size)
print(left[left.nonzero()])
print(right.nonzero()[0].size)
print(right[right.nonzero()])

1182
[-0.70710678 -0.0625     -0.0625     ... -0.70710678 -0.5
  0.5       ]
195
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
